In [14]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps

#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from AAC import AnimalShelter





###########################
# Data Manipulation / Model
###########################
# FIX ME change for your username and password and CRUD Python module name
username = "aacuser"
password = "password"
shelter = AnimalShelter()
shelter._init_(username,password)


# class read method must support return of cursor object 
df = pd.DataFrame.from_records(dumps(shelter.read({})))
dff =pd.DataFrame.from_records(dumps(shelter.read({}))) ## For map and graph


#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

#FIX ME Add in Grazioso Salvare’s logo
image_filename = '/home/16514911_snhu/Downloads/Grazioso Salvare Logo.jpg' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard'))),
    html.Hr(),
    html.Div(
        html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode())), #Logo
        html.Hr(),
        html.Center(html.H6("Jacqueline Woods")), #unique handle

         html.Div(className ='row',
                children=[
                html.Button(id='submit-button-one',n_clicks=0, children='Cats'), #Filter options
                html.Button(id='submit-button-two',n_clicks=0,children='Dogs'),
                html.Button(id='submit-button-three',n_clicks=0, children='WaterDogs'), #Filter options
                html.Button(id='submit-button-four',n_clicks=0,children='Mountain or Wildernesss'),
                html.Button(id='submit-button-five',n_clicks=0, children='Disaster or Tracking')] #Filter option)
                
    ),
    html.Hr(),
    dt.DataTable(
        id='datatable-interactivity',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        
    ),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
)])

#############################################
# Interaction Between Components / Controller
#############################################

#Buttons work to filter dashboard              
@app.callback(Output('datatable-interactivity',"data"),
              [Input('submit-button-one','n_clicks'),
              Input('submit-button-two','n_clicks'),
              Input('submit-button-three','n_clicks'),
              Input('submit-button-four','n_clicks'),
              Input('submit-button-five','n_clicks')])
def on_click(bt1, bt2,bt3,bt4,bt5):  #button method
    if(int(bt1) ==0 and int(bt2) ==0):
        df =pd.DataFrame.from_records(shelter.readAll({}))
   
    elif (int(bt1) > int(bt2)):
        df = pd.DataFrame(list(shelter.read({"animal__type":"Cat"})))
    else:
        df =pd.DataFrame(list(shelter.read({"animal_type":"Dog"})))
    
    if(int(bt3) ==0 and int(bt4) ==0 and int(bt5)):
        df =pd.DataFrame.from_records(shelter.readAll({}))
    
    elif ((int(bt3) > int(bt4) and (int(bt3)>int(bt5)))): # Water rescue query
            df = pd.DataFrame(list(dumps(shelter.read({"breed":"Labrador Retriever Mix"},
                                                {"breed": "Chesapeak Bay Retriever"},{"breed":"NewFoundland"},
                                                {"sex_upon_outcome":"Intact Female"} ,
                                                {'$or':[{"age_upon_outcome":{'$lt':156}}, {"age_upon_outcome":{'$gt':26}}]}))))
    elif((int(bt4) > int(bt3)) and (int(bt4)>int(bt5))): #Mountain rescue
            df = pd.DataFrame(dumps(list(shelter.read({"breed":"German Shepard"},
                                                {"breed": "Alaskan Malamute"},{"breed":"Old English Sheepdog"},
                                                {"breed":"Siberian Huskey"},{"breed":"Rottweiler"},
                                                {"sex_upon_outcome":"Intact Male"} ,
                                                {'$or':[{"age_upon_outcome":{'$lt':156}}, {"age_upon_outcome":{'$gt':26}}]}))))

    elif((int(bt5) > int(bt4)) and (int(bt5)>int(bt3))): #Disaster
            df = pd.DataFrame(list(dumps(shelter.read({"breed":"German Shepard"},
                                                {"breed": "Doberman Pinscher"},{"breed":"Golden Retriever"},
                                                {"breed":"Blood Hound"},{"breed":"Rottweiler"},
                                                {"sex_upon_outcome":"Intact Male"} ,
                                                {'$or':[{"age_upon_outcome":{'$lt':300}}, {"age_upon_outcome":{'$gt':20}}]}))))
    else:
        df.pd.DataFrame(list(dumps(shelter.readAll({}))))
        

        return df.to_dict('records')




@app.callback(
    Output('datatable-interactivity', 'style_data_conditional'),
    [Input('datatable-interactivity', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-interactivity', "derived_viewport_data")])
def update_graphs(viewData):
    dff= pd.DataFrame(allData)
    pieChart = px.pie(
        data_frame=dff,
        names=dff['outcome_type'],
        values = dff['breed'],
        hole = .3,
    )
    return pieChart


@app.callback(
    Output('map-id', "children"),
    [Input('datatable-interactivity', 'derived_viewport_data'),
     Input('datatable-interactivity', 'derived_virtual_selected_rows')
    ])
def update_map(viewData):
    dff = pd.DataFrame.from_dict(viewData)
    dff = df if viewData is None else pd.DataFrame(viewData)
    selected_animal = None
    if not derived_virtual_selected_rows:
        selected_animal = dff.iloc[0]
    else:
        selected_animal = dff.iloc[derived_virtual_selected_rows[0]]
    latitude = selected_animal[12]
    longitude =selected_animal[13]
    breed = selected_animal[3]
    name = selected_animal[0]
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            dl.Marker(position=[latitude,longitude], children=[
                dl.Tooltip(breed),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(name)
                ])
            ])
        ])
    ]



app

TypeError: unhashable type: 'dict'